In [1]:
import torch
from transformers import pipeline

In [8]:
# Define a function to split the long description into manageable chunks
def split_text(text, max_tokens):
    """Split the input text into chunks with a maximum token length."""
    words = text.split()
    chunks = []
    current_chunk = []

    # Accumulate words until we reach the token limit
    for word in words:
        current_chunk.append(word)
        if len(current_chunk) > max_tokens:
            chunks.append(" ".join(current_chunk[:-1]))  # Add current chunk (excluding last word)
            current_chunk = [current_chunk[-1]]  # Start a new chunk with the last word
    chunks.append(" ".join(current_chunk))  # Add the last chunk

    return chunks

In [3]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"

In [4]:
# Initialize the text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,  # If you have GPU support
    device_map="auto",  # Automatically uses available GPU
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# Example patent claim (Claim from Patent 1)
claim_patent_1 = """
1. A method for wireless communication comprising: receiving a signal at a base station; processing the received signal 
   for decoding; and transmitting a decoded signal to a user equipment (UE).
"""

# Long patent description (Patent 2) - split this into chunks
description_patent_2 = """
The base station performs signal processing using advanced modulation techniques, such as Quadrature Amplitude Modulation (QAM),
to enhance the data rate. It is capable of handling high-speed data transmission, especially for urban areas with high traffic density.
The user equipment (UE) is designed to support 5G NR (New Radio) frequencies and provides high throughput with low latency, even in crowded environments.
Additional capabilities include supporting massive MIMO (Multiple Input Multiple Output) antennas for beamforming to increase network efficiency.
Furthermore, the system can integrate with IoT (Internet of Things) devices for low-power communication in smart cities.
"""

In [13]:
# Create the instruction template
instruction_template = """
Based on the following claim from Patent 1, generate a new patent claim that incorporates the relevant features mentioned in the description of Patent 2.

Claim from Patent 1: {claim}

Description from Patent 2: {chunk}

Print only the new claim with no notes or comments, just a raw claim with 450 words max. If you don't find any relevant features ignore the current chunk. 
"""


In [ ]:
# Split the long patent description into smaller chunks
description_chunks = split_text(description_patent_2, max_tokens=512)

In [ ]:
# Process each chunk and generate new claims
generated_claims = []
for chunk in description_chunks:
    instruction = instruction_template.format(claim=claim_patent_1, chunk=chunk)
    outputs = pipe(instruction, max_new_tokens=500)  # Limit output length to prevent excessive text
    generated_text = outputs[0]["generated_text"]

    # Try to remove the instruction and description from the generated output
    # Assuming that the output is a continuation of the instruction, we will strip it
    new_claim = generated_text.replace(instruction, "").strip()  # Remove the instruction part

    # Append only the new claim to the list
    generated_claims.append(new_claim)

# Combine the generated claims from each chunk
final_claim = "\n".join(generated_claims)
print(final_claim) 

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1. A method for wireless communication comprising: receiving a signal at a base station; processing the received signal for decoding; transmitting a decoded signal to a user equipment (UE); and employing advanced modulation techniques such as Quadrature Amplitude Modulation (QAM) to enhance data rate, particularly in urban areas with high traffic density; and utilizing the user equipment (UE) to provide high throughput with low latency, even in crowded environments; and supporting massive MIMO (Multiple Input Multiple Output) antennas for beamforming to increase network efficiency; and integrating with IoT (Internet of Things) devices for low-power communication in smart cities.
